<p style="text-align:center">
  <a href="https://www.linkedin.com/company/100622063" target="_blank" title="Follow LevelUp360 on LinkedIn">
    <img src="../../assets/levelup360-inverted-logo-transparent.svg" alt="LevelUp360" width="220">
  </a>
</p>

# Marketing Team – Week 05: Minimal Microsoft Agent Framework Validation

This notebook validates whether by adopting Microsoft Agent Framework we can preserve `LLMClient` features: cost tracking, model-agnostic interface, and OpenAI-native tool-calling.

---

## Environment Setup

### Prerequisites
- Python 3.10+
- `.env` with API keys (see below)
- Virtual environment in repo root
- VS Code Jupyter extension (or JupyterLab)

### Required Environment Variables
    # OpenRouter
    OPENROUTER_API_KEY=sk-...

    # Azure (optional for embeddings)
    AZURE_OPENAI_ENDPOINT=https://...
    AZURE_OPENAI_API_KEY=...
    AZURE_OPENAI_API_VERSION=...

    # Web search
    TAVILY_API_KEY=...

    # LangSmith (optional tracing)
    LANGSMITH_TRACING_V2=true
    LANGSMITH_API_KEY=...
    LANGSMITH_PROJECT=levelup360-marketing-team
    LANGSMITH_ENDPOINT=https://api.smith.langchain.com

    # (Optional) Pricing for local cost tracking
    GPT4O_INPUT_PRICE_PER_1K=...
    GPT4O_OUTPUT_PRICE_PER_1K=...
    EMBEDDING_PRICE_PER_1K=...
    TAVILY_PRICE_PER_CALL=...

### One-Time Setup (PowerShell on Windows)
    # From workspace root:
    python -m venv .venv
    .\\.venv\\Scripts\\Activate.ps1

    # If activation blocked, run once (PowerShell):
    Set-ExecutionPolicy -Scope CurrentUser RemoteSigned

    # Upgrade tooling
    python -m pip install --upgrade pip setuptools wheel

    # Install dependencies
    pip install -r requirements.txt
    
    # Editable install (so `from src...` works everywhere)
    # Ensure pyproject.toml exists at the project root where src/ lives:
    #   [build-system]
    #   requires = ["setuptools>=68", "wheel"]
    #   build-backend = "setuptools.build_meta"
    #
    #   [project]
    #   name = "marketing-team"
    #   version = "0.1.0"
    #   requires-python = ">=3.10"
    #
    #   [tool.setuptools.packages.find]
    #   where = ["."]
    #   include = ["src*"]
    pip install -e .

    # (Optional) Register kernel
    python -m ipykernel install --user --name marketing-team --display-name "Python (marketing-team)"

### One-Time Setup (macOS/Linux bash)
    # From workspace root:
    python -m venv .venv
    source .venv/bin/activate

    # Upgrade tooling
    python -m pip install --upgrade pip setuptools wheel

    # Install dependencies
    pip install -r requirements.txt

    # Editable install (-e) so `from src...` imports work in scripts/notebooks/tests
    pip install -e .

    # (Optional) Register kernel
    python -m ipykernel install --user --name marketing-team --display-name "Python (marketing-team)"

### Verify Installation
    # Should print "OK" without ImportError
    python -c "import src; from src.agents.graph import build_content_workflow; print('OK')"

---

## Notebook Flow

1. **Setup** - Environment setup + `LLMClient` selection (`openrouter`).
2. **Single agent test** - Run a single agent test with a simple prompt; prints cost and output.
3. **Two-agent state pass** - Tun a two-agent state pass test (`ResearchAgent` → `WriterAgent`); prints both outputs.
---

In [1]:
import os
import nest_asyncio
from agent_framework import BaseAgent, AgentRunResponse, ChatMessage
from src.infrastructure.llm.llm_client import LLMClient
import rich as rprint
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGSMITH_TRACING"] = "false"
nest_asyncio.apply()

completion_client = LLMClient()
completion_client.get_client("openrouter")

2025-11-17 11:53:27,994 - src.infrastructure.llm.llm_client - WARNING - Using default price for CLAUDE_SONET_4_INPUT_PRICE_PER_1K: 0.003000 EUR/1K tokens
[2025-11-17 11:53:27 - C:\Users\mtomas\OneDrive\Coding\LevelUp360\Builder-repos\Agentic-AI\agentic-ai-examples\marketing-team\src\infrastructure\llm\llm_client.py:243 - WARNING] Using default price for CLAUDE_SONET_4_INPUT_PRICE_PER_1K: 0.003000 EUR/1K tokens
2025-11-17 11:53:28,002 - src.infrastructure.llm.llm_client - WARNING - Using default price for CLAUDE_SONET_4_OUTPUT_PRICE_PER_1K: 0.015000 EUR/1K tokens
[2025-11-17 11:53:28 - C:\Users\mtomas\OneDrive\Coding\LevelUp360\Builder-repos\Agentic-AI\agentic-ai-examples\marketing-team\src\infrastructure\llm\llm_client.py:243 - WARNING] Using default price for CLAUDE_SONET_4_OUTPUT_PRICE_PER_1K: 0.015000 EUR/1K tokens
2025-11-17 11:53:28,005 - src.infrastructure.llm.llm_client - WARNING - Using default price for GPT5_INPUT_PRICE_PER_1K: 0.001500 EUR/1K tokens
[2025-11-17 11:53:28 - C:\

In [2]:
#Test single agent

class TestAgent(BaseAgent):
    def __init__(self, llm_client: LLMClient):
        super().__init__()
        self.llm = llm_client

    async def run(self, messages, thread=None, **kwargs):
        user_prompt = "Generate a content strategy for a tech startup"
        response = self.llm.get_completion(
            model="openai/gpt-4o-mini",
            messages=[{"role": "user", "content": user_prompt}],
            temperature=0.5,
        )
        try:
            print(f"Call cost (EUR): {response.cost:.6f}")
        except Exception:
            pass
        msg_text = getattr(response, "content", None) or str(response)
        result = AgentRunResponse(messages=[ChatMessage(role="assistant", text=msg_text)])
        print(result.messages[0].text)
        return result
    
async def main():
    agent = TestAgent(completion_client)
    result = await agent.run(messages=[])
    first = result.messages[0] if getattr(result, "messages", None) else None
    print(first.text if first else "No messages returned from agent.")

# Run the agent
await main()

[2025-11-17 11:53:32 - c:\Users\mtomas\OneDrive\Coding\LevelUp360\Builder-repos\Agentic-AI\agentic-ai-examples\.venv\Lib\site-packages\langsmith\client.py:2284 - WARNING] Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


Call cost (EUR): 0.000540
Creating a content strategy for a tech startup involves understanding your target audience, defining your brand voice, and establishing clear goals. Below is a comprehensive content strategy tailored for a tech startup:

### 1. Define Your Goals
- **Brand Awareness**: Increase visibility within your target market.
- **Lead Generation**: Capture leads through informative content.
- **Customer Education**: Help users understand your product and its benefits.
- **Community Building**: Foster a community around your brand and tech niche.

### 2. Identify Your Target Audience
- **Demographics**: Age, gender, location, education, job roles (e.g., tech enthusiasts, business decision-makers, developers).
- **Psychographics**: Interests, challenges, and pain points related to technology.
- **Buyer Personas**: Create detailed personas for different segments of your audience.

### 3. Content Pillars
Establish key themes that align with your startup's mission and audience

In [3]:
# Test state passing between agents

class ResearchAgent(BaseAgent):
    def __init__(self, llm_client):
        super().__init__()
        self.llm = llm_client

    async def run(self, messages, thread=None, **kwargs):
        topic = kwargs.get("topic", "unknown")
        response = self.llm.get_completion(
            model="openai/gpt-4o-mini",
            messages=[{"role": "user", "content": f"Research key points about: {topic}"}],
            temperature=0.5,
        )
        msg_text = getattr(response, "content", None) or str(response)
        return AgentRunResponse(messages=[ChatMessage(role="assistant", text=msg_text)])

class WriterAgent(BaseAgent):
    def __init__(self, llm_client):
        super().__init__()
        self.llm = llm_client

    async def run(self, messages, thread=None, **kwargs):
        research = messages[-1].text if messages else "No research"
        response = self.llm.get_completion(
            model="openai/gpt-4o-mini",
            messages=[{"role": "user", "content": f"Write content based on: {research}"}],
            temperature=0.7,
        )
        msg_text = getattr(response, "content", None) or str(response)
        return AgentRunResponse(messages=[ChatMessage(role="assistant", text=msg_text)])

# Sequential flow test
researcher = ResearchAgent(completion_client)
writer = WriterAgent(completion_client)

research_result = await researcher.run(messages=[], topic="% of AI projects that fail, and why they fail")
writer_result = await writer.run(messages=research_result.messages)

print("Research:", research_result.messages[0].text)
print("\nContent:", writer_result.messages[0].text)

[2025-11-17 11:53:49 - c:\Users\mtomas\OneDrive\Coding\LevelUp360\Builder-repos\Agentic-AI\agentic-ai-examples\.venv\Lib\site-packages\langsmith\client.py:2352 - WARNING] Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
[2025-11-17 11:54:02 - c:\Users\mtomas\OneDrive\Coding\LevelUp360\Builder-repos\Agentic-AI\agentic-ai-examples\.venv\Lib\site-packages\langsmith\client.py:2352 - WARNING] Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


Research: The success and failure rates of AI projects have been a topic of extensive research and discussion in the tech industry. Here are some key points regarding the percentage of AI projects that fail and the reasons behind these failures:

### Percentage of AI Projects That Fail

1. **High Failure Rate**: Various studies suggest that between 70% to 90% of AI projects fail to deliver the expected results or achieve their intended goals. This high failure rate is often attributed to several factors, including unrealistic expectations, lack of proper planning, and insufficient expertise.

2. **Gartner Research**: According to a 2020 report by Gartner, approximately 85% of AI projects will deliver erroneous outcomes due to bias in data, algorithms, or the teams responsible for managing them.

3. **McKinsey Report**: A McKinsey Global Institute report indicated that while 50% of organizations have adopted AI in at least one business function, only 10% of those organizations achieve s

[2025-11-17 11:54:14 - c:\Users\mtomas\OneDrive\Coding\LevelUp360\Builder-repos\Agentic-AI\agentic-ai-examples\.venv\Lib\site-packages\langsmith\client.py:2352 - WARNING] Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
